In [1]:
import pandas as pd
import numpy as np
import glob
import os

from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

In [2]:
# Load in the summary data
path = os.getcwd()
path = path + '\Construction_Data\\'

df = pd.read_excel(path + 'summary_data.xlsx')

In [3]:
df.head()

,project,wbs,sub,units,measurement_unit,windows,doors,holes,corners,wall_count,isolation,support,roof,outer,inner,mortar,outer_perc,inner_perc,mortar_perc,hours
0,2,outer walls,basement,76.970,m2,0.0,2.0,2.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,590.0
1,2,outer walls,1st floor,229.254,m2,51.0,6.0,57.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1396.0
2,2,outer walls,2nd floor,202.704,m2,82.0,9.0,91.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1056.0
3,2,outer walls,3d floor,105.122,m2,22.0,3.0,25.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,848.0
4,2,outer walls,4th floor,105.122,m2,22.0,3.0,25.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,558.0


In [4]:
# Defina a function that takes in the test set and predictions and calculates mean absolute percentage error, MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Rough estimation based on project 1
First step was to create the rough estimation where the ratio of hours per unit in project 1 was used to estimate the duration in project 2.

In [5]:
newdf = pd.DataFrame()

# Loop through sub-dataframes for each WBS item
for wbs, df_wbs in df.groupby('wbs'):
    
    # If project 1 has data on that WBS item, then proceed
    if (df_wbs['project'] == 1).any():
        temp = df_wbs.copy()
        # Calculate the ratio for that WBS item from the project 1 data
        p1_row = temp[temp['project'] == 1]
        hour_per_unit = p1_row['hours'] / p1_row['units']
        
        # Create predictions and calculate errors for every row in the dataset
        temp['predicions'] = float(hour_per_unit) * temp['units']
        temp['error'] = temp['hours'] - temp['predicions']
        
        # Add to a new dataframe
        newdf = pd.concat([newdf, temp])

In [6]:
# Remove the lines for project 1 so the perfect predictions are not included 
newdf = newdf[newdf['project'] != 1]

In [7]:
errorlist = list()

# Remove rows with missing predictions
curr_df = newdf[newdf['error'].notnull()]

# Loop through sub-dataframes for each WBS item
for wbs, df_wbs in curr_df.groupby('wbs'):
    
    # Calculate MAPE for each WBS item
    mape = mean_absolute_percentage_error(df_wbs['hours'], df_wbs['predicions'])
    
    # Append the WBS item and MAPE to a list
    errorlist.append([wbs, mape])

# Calculate and append the overall accuracy with MAPE
overall_mape = mean_absolute_percentage_error(curr_df['hours'], curr_df['predicions'])
errorlist.append(['Overall', overall_mape])

# Transform into a dataframe to combine later
benchmark_errors = pd.DataFrame(errorlist, columns = ['WBS Item', 'Ratio Estimation'])

# Transform into a dataframe for presentation purposes
benchmark_errors_print = pd.DataFrame(errorlist, columns = ['WBS Item', 'MAPE'])
benchmark_errors_print['MAPE'] = benchmark_errors_print['MAPE'].round(2)
benchmark_errors_print

,WBS Item,MAPE
0,ceilings,50.90
1,concrete floor,43.73
2,doors inside,778.86
3,floors,85.56
4,inner walls,112.53
5,paint inside,79.59
6,Overall,101.28


# CV estimation on project 2
The next part only uses information from project 2 to estimate the duration of each WBS item. Here there is more detail to work with so the experiment is to check if there is any accuracy gain from increased granularity.

In [8]:
# Create a function that takes in a dataframe and a machine learning model an does leave-one-out cross validation
def run_CV(dataframe, model):
    prediction_list = list()
    actual_list = list()
    
    # Prepare cross validation based on the total number of rows
    kfold = KFold(len(dataframe), shuffle = True, random_state = 42)
    # Go through each possible combination of the cross validation
    for train, test in kfold.split(dataframe):
        # Separate into training sets
        x_train = dataframe.iloc[train].drop(columns = 'hours')
        y_train = dataframe.iloc[train]['hours']
        
        # Separate into test sets
        x_test = dataframe.iloc[test].drop(columns = 'hours')
        y_test = dataframe.iloc[test]['hours']
        
        # Get the predictions based on the input model
        y_pred = model(x_train, x_test, y_train)
        
        # Store the predictions and corrisponding test set
        prediction_list.append(y_pred)
        actual_list.append(y_test.values)
    
    # Calculate and return the cross validation MAPE, along with the predictions and actuals
    error = mean_absolute_percentage_error(actual_list, prediction_list)    
    return (error, prediction_list, actual_list)

In [9]:
# Create a function that fits and predicts using linear regression
def linear_regression(x_train, x_test, y_train):
    model = LinearRegression().fit(x_train, y_train)
    y_pred = model.predict(x_test)
    
    return y_pred

In [10]:
# Create a function that fits and predicts using regression tree
def regression_tree(x_train, x_test, y_train):
    model = DecisionTreeRegressor().fit(x_train, y_train)
    y_pred = model.predict(x_test)
    
    return y_pred

In [11]:
# Create a function that fits and predicts using random forest
def random_forest(x_train, x_test, y_train):
    model = RandomForestRegressor(n_estimators = 100).fit(x_train, y_train)
    y_pred = model.predict(x_test)
    
    return y_pred

In [12]:
# Create a function that fits and predicts using extreme gradient boosting
def xgboost(x_train, x_test, y_train):
    model = xgb.XGBRegressor(objective = 'reg:squarederror').fit(x_train, list(y_train))
    y_pred = model.predict(x_test)
    
    return y_pred

In [13]:
# Remove rows from project 1 and WBS items that no additional features could be added
p2_df = df[(df['project'] != 1) & (df['wbs'] != 'doors inside') & (df['wbs'] != 'windows')]

# Initalise the new dataframe with a single column with all WBS items along with Overall
model_df = pd.DataFrame(np.append(p2_df['wbs'].unique(), 'Overall'))

# Define a list of all the models that will be used
model_list = [linear_regression, regression_tree, random_forest, xgboost]
model_names = ['Linear Regression', 'Regression Tree', 'Random Forest', 'XGBoost']

# Loop through each model
for model in model_list:
    
    errors = list()
    overall_predictions = list()
    overall_actuals = list()
    
    # Loop through each WBS item
    for wbs, df_wbs in p2_df.groupby('wbs'):
        
        # Drop columns and rows that the models cannot handle, such as categorical or NaN values
        df_wbs = df_wbs.drop(columns = ['project', 'wbs', 'sub', 'measurement_unit'])
        df_wbs = df_wbs.dropna(axis = 'columns', how = 'all')
        df_wbs = df_wbs.dropna(axis = 'rows', how = 'any')

        # Get the MAPE, predictions and test set for every WBS item
        wbs_error, temp_pred, temp_actual = run_CV(df_wbs, model)

        # Append the WBS item and corrisponding error to a list of lists
        errors.append([wbs, wbs_error])
        
        # Combine the predictino and test lists
        overall_predictions = overall_predictions + temp_pred
        overall_actuals = overall_actuals + temp_actual

    # Append a final Overall category and the MAPE calculated from the overall predictions and test set
    errors.append(['Overall', mean_absolute_percentage_error(overall_actuals, overall_predictions)]) 
    # Combine into a dataframe that includes all errors for that specific model
    error_df = pd.DataFrame(errors)
    
    # Merge the model error dataframe to the master dataframe
    model_df = pd.merge(model_df, error_df, on = 0)

# Rename the columns for presentation
model_df.columns = ['WBS Item'] + model_names

In [14]:
# Merge the new dataframe with the original dataframe from the project 1 experiment
final_df = pd.merge(model_df, benchmark_errors, on = 'WBS Item', how = 'outer')

# Move overall row to the bottom for presentation
overall_row = final_df.loc[final_df[final_df['WBS Item'] == 'Overall'].index]
final_df = final_df.drop(final_df[final_df['WBS Item'] == 'Overall'].index)
final_df.append(overall_row).reset_index(drop = 'True').round(2)

,WBS Item,Linear Regression,Regression Tree,Random Forest,XGBoost,Ratio Estimation
0,outer walls,26.54,39.05,28.11,32.28,NaN
1,floors,73.05,38.01,42.83,37.84,85.56
2,ceilings,42.70,43.61,37.23,43.42,50.90
3,concrete floor,254.21,57.11,39.22,43.18,43.73
4,inner walls,466.95,97.48,88.44,126.32,112.53
5,glass walls,7.12,32.66,24.48,24.83,NaN
6,paint inside,473.20,260.04,342.36,259.73,79.59
7,doors inside,NaN,NaN,NaN,NaN,778.86
8,Overall,203.63,82.96,87.97,82.73,101.28
